In [134]:
import pandas as pd

# Load the CSV
df = pd.read_csv('donnees_vol_mitm.csv')  # use your actual filename if different
df['is_attack'] = 0
# List of columns to keep
columns_to_keep = [
    "mission_id", "time", "real_x", "real_y", "real_z",
    "target_x", "target_y", "target_z",
    "vx", "vy", "vz", "ax", "ay", "az",
    "roll", "pitch", "yaw",
    "wind_x", "wind_y", "wind_z",
    "erreur_m", "velocity", "rain", "fog", "snow",
    "is_attack"      

]


filtered_df = df[columns_to_keep]


print(" Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'")

 Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'


In [135]:
print(df.isna().any().any())
colonnes_avec_nan = df.columns[df.isna().any()].tolist()

# Afficher les colonnes concernées
print("Colonnes contenant des NaN :")
print(colonnes_avec_nan)

# Afficher le nombre de NaN par colonne
print("\nNombre de NaN par colonne :")
print(df[colonnes_avec_nan].isna().sum())

True
Colonnes contenant des NaN :
['snow']

Nombre de NaN par colonne :
snow    3537
dtype: int64


In [136]:
import pandas as pd

# Charger le fichier CSV
df = filtered_df  # Remplace par ton vrai nom de fichier

# Afficher les colonnes contenant des NaN avant traitement
print("🔍 Colonnes avec NaN AVANT traitement :")
print(df.isna().sum()[df.isna().sum() > 0])

# === MÉTHODE DE REMPLISSAGE ===
# Option 1 : Remplir avec la moyenne de chaque colonne numérique
df = df.fillna(df.mean(numeric_only=True))

# Option 2 (alternative) : remplir avec 0 — décommente si tu préfères
# df = df.fillna(0)

# Option 3 (alternative) : remplir avec la médiane
# df = df.fillna(df.median(numeric_only=True))

# Afficher les colonnes contenant encore des NaN après traitement
print("\n✅ Colonnes avec NaN APRÈS traitement :")
print(df.isna().sum()[df.isna().sum() > 0])

# Sauvegarder le DataFrame propre
print("\n✅ Fichier nettoyé enregistré sous 'fichier_sans_nan.csv'")


🔍 Colonnes avec NaN AVANT traitement :
snow    3537
dtype: int64

✅ Colonnes avec NaN APRÈS traitement :
Series([], dtype: int64)

✅ Fichier nettoyé enregistré sous 'fichier_sans_nan.csv'


In [137]:
import pandas as pd

# Load the CSV file
df = filtered_df

# Final list of valid flight rows
output_rows = []

# Track current block of rows
current_block = []
current_id = None

# Counter for assigning new clean mission_ids
new_flight_id = 1

# Go through rows one by one
for _, row in df.iterrows():
    row_id = row['mission_id']

    if current_id is None:
        current_id = row_id
        current_block.append(row)
    elif row_id == current_id:
        current_block.append(row)
    else:
        # ID changed, time to process the current block
        if len(current_block) == 21:
            for r in current_block:
                r['mission_id'] = new_flight_id
                output_rows.append(r)
            new_flight_id += 1

        # Start a new block
        current_block = [row]
        current_id = row_id

# Handle the last block
if len(current_block) == 21:
    for r in current_block:
        r['mission_id'] = new_flight_id
        output_rows.append(r)

# Convert to DataFrame
output_df = pd.DataFrame(output_rows)


print("✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'")

✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'


In [138]:
print(df.isna().any().any())
colonnes_avec_nan = df.columns[df.isna().any()].tolist()

# Afficher les colonnes concernées
print("Colonnes contenant des NaN :")
print(colonnes_avec_nan)

# Afficher le nombre de NaN par colonne
print("\nNombre de NaN par colonne :")
print(df[colonnes_avec_nan].isna().sum())

True
Colonnes contenant des NaN :
['snow']

Nombre de NaN par colonne :
snow    3537
dtype: int64


In [139]:
import pandas as pd
import numpy as np

# Charger les données
df = output_df

# Vérifier que les colonnes nécessaires existent
colonnes_requises = ["real_x", "real_y", "real_z", "target_x", "target_y", "target_z"]
for col in colonnes_requises:
    if col not in df.columns:
        raise ValueError(f"La colonne '{col}' est manquante dans le fichier CSV.")


df["erreur_m"] = np.sqrt(
    (df["real_x"] - df["target_x"])**2 +
    (df["real_y"] - df["target_y"])**2 +
    (df["real_z"] - df["target_z"])**2
)


df.to_csv("donnees_vol_multiple_filtered_R.csv", index=False)
print("✅ 'erreur_m' recalculée et sauvegardée dans 'donnees_vols_avec_erreur_calculee.csv'")


✅ 'erreur_m' recalculée et sauvegardée dans 'donnees_vols_avec_erreur_calculee.csv'


In [140]:
import pandas as pd

# Charger les données
df = output_df
df["erreur_m"] = pd.to_numeric(df["erreur_m"], errors="coerce")

# Liste des vols valides
vols_valides = []

# Vérification des conditions sur chaque vol
for mission_id, groupe in df.groupby("mission_id"):
    groupe_sorted = groupe.sort_values("time")
    premiers_wp = groupe_sorted.head(6)

    moyenne_vol = groupe_sorted["erreur_m"].round(2).mean()
    condition_moyenne = moyenne_vol >= 15


    # Appliquer le OU : on supprime le vol si l'une des deux conditions est vraie
    if  condition_moyenne:
        continue

    # Sinon, garder ce vol
    vols_valides.append(mission_id)

# Filtrer le DataFrame avec les vols valides
df_final = df[df["mission_id"].isin(vols_valides)]

# Sauvegarder dans un nouveau fichier CSV
df_final.to_csv("donnees_vols_valides_Rate.csv", index=False)

# Affichage de la moyenne d'erreur et du nombre de vols gardés
moyenne_finale = df_final["erreur_m"].mean()
print(f"Moyenne finale des erreurs : {moyenne_finale:.2f} mètres")
print(f"Nombre de vols restants : {len(vols_valides)}")
print("Fichier 'donnees_vols_valides_Rate.csv' enregistré avec succès.")


Moyenne finale des erreurs : 9.69 mètres
Nombre de vols restants : 267
Fichier 'donnees_vols_valides_Rate.csv' enregistré avec succès.


In [141]:
condition = df_final['erreur_m'] > 2.5
df_final.loc[condition, ['real_x', 'real_y', 'real_z']] = df_final.loc[condition, ['target_x', 'target_y', 'target_z']].values
print(f"\n✅ Correction appliquée à {condition.sum()} points avec erreur > 2.5m")
print(f"✅ Shape des données corrigées: {df_final.loc[condition, ['real_x', 'real_y', 'real_z']].shape}")
df_final.to_csv("donnees_vols_valides_Rate.csv", index=False)


✅ Correction appliquée à 5121 points avec erreur > 2.5m
✅ Shape des données corrigées: (5121, 3)


In [150]:

print(df.isna().any().any())
colonnes_avec_nan = df.columns[df.isna().any()].tolist()

# Afficher les colonnes concernées
print("Colonnes contenant des NaN :")
print(colonnes_avec_nan)

# Afficher le nombre de NaN par colonne
print("\nNombre de NaN par colonne :")
print(df[colonnes_avec_nan].isna().sum())

True
Colonnes contenant des NaN :
['snow']

Nombre de NaN par colonne :
snow    735
dtype: int64


In [143]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""""
import pandas as pd
import numpy as np

def main():
    # 1. Charger les données
    df = pd.read_csv('donnees_vols_valides_R.csv')

    # 2. Calcul de la matrice de corrélation
    corr = df.corr()
    print("Matrice de corrélation :")
    print(corr)
    print("\n" + "="*80 + "\n")

    # 3. Extraction des paires de variables (sans doublons ni diagonale)
    mask = np.triu(np.ones(corr.shape), k=1).astype(bool)
    corr_pairs = (
        corr.where(mask)
            .stack()
            .reset_index()
            .rename(columns={'level_0': 'Variable1', 'level_1': 'Variable2', 0: 'Corrélation'})
    )

    # 4. Filtrage des paires avec corrélation >= 0.7 en valeur absolue
    high_corr = corr_pairs[corr_pairs['Corrélation'].abs() >= 0.7] \
        .sort_values(by='Corrélation', ascending=False)

    # 5. Affichage des paires à forte corrélation
    print("Paires de variables avec |corrélation| ≥ 0.7 :")
    for _, row in high_corr.iterrows():
        v1, v2, r = row['Variable1'], row['Variable2'], row['Corrélation']
        print(f"  - {v1} ↔ {v2} : corrélation = {r:.3f}")

if __name__ == "__main__":
    main()"""


'""\nimport pandas as pd\nimport numpy as np\n\ndef main():\n    # 1. Charger les données\n    df = pd.read_csv(\'donnees_vols_valides_R.csv\')\n\n    # 2. Calcul de la matrice de corrélation\n    corr = df.corr()\n    print("Matrice de corrélation :")\n    print(corr)\n    print("\n" + "="*80 + "\n")\n\n    # 3. Extraction des paires de variables (sans doublons ni diagonale)\n    mask = np.triu(np.ones(corr.shape), k=1).astype(bool)\n    corr_pairs = (\n        corr.where(mask)\n            .stack()\n            .reset_index()\n            .rename(columns={\'level_0\': \'Variable1\', \'level_1\': \'Variable2\', 0: \'Corrélation\'})\n    )\n\n    # 4. Filtrage des paires avec corrélation >= 0.7 en valeur absolue\n    high_corr = corr_pairs[corr_pairs[\'Corrélation\'].abs() >= 0.7]         .sort_values(by=\'Corrélation\', ascending=False)\n\n    # 5. Affichage des paires à forte corrélation\n    print("Paires de variables avec |corrélation| ≥ 0.7 :")\n    for _, row in high_corr.iterrow

In [144]:
import pandas as pd

# Charger les données
df = pd.read_csv('donnees_vols_valides_Rate.csv')
df = df.reset_index(drop=True)

# Ajouter l'index du waypoint par mission
df['waypoint_id'] = df.groupby('mission_id').cumcount()

# Regrouper les waypoints par paquets de 7 (3 groupes pour 21 waypoints)
df['groupe_id'] = df['waypoint_id'] // 7
df['groupe_unique_id'] = df['mission_id'].astype(str) + '_' + df['groupe_id'].astype(str)

# Colonnes à pivoter (on exclut les identifiants)
cols_to_pivot = [col for col in df.columns if col not in ['mission_id', 'is_attack']]

# Stocker les résultats
result_dfs = []

# Utiliser groupby() au lieu d'une boucle sur les valeurs uniques
for (mission_id, groupe_id), groupe_df in df.groupby(['mission_id', 'groupe_id']):
    if len(groupe_df) > 0:
        is_attack = groupe_df['is_attack'].iloc[0]
        groupe_dict = {'mission_id': mission_id, 'groupe_id': groupe_id, 'is_attack': is_attack}

        for col in cols_to_pivot:
            for i, (_, row) in enumerate(groupe_df.iterrows()):
                if i < 7:
                    groupe_dict[f"{col}_{i+1}"] = row[col]
                else:
                    break

        result_dfs.append(groupe_dict)

# Convertir en DataFrame
df_grouped = pd.DataFrame(result_dfs)

# Trier par mission et groupe
df_grouped = df_grouped.sort_values(['mission_id', 'groupe_id']).reset_index(drop=True)

cols = [col for col in df_grouped.columns if col != 'is_attack'] + ['is_attack']
df_grouped = df_grouped[cols]

df_grouped.to_csv("vols_groupes_par_7.csv", index=False)

cols_to_drop = [col for col in df_grouped.columns if col.startswith(('groupe_id', 'groupe_unique_id', 'waypoint_id')) and col != 'mission_id']
df_grouped_clean = df_grouped.drop(columns=cols_to_drop)

df_grouped.to_csv("vols_groupes_par_7.csv", index=False)

# Affichage
print(f"✅ Groupement terminé. {len(df_grouped_clean)} groupes créés à partir de {df['mission_id'].nunique()} missions.")
print(f"✅ Colonnes supprimées : {cols_to_drop}")
print(df_grouped_clean.head(2).T)

✅ Groupement terminé. 801 groupes créés à partir de 267 missions.
✅ Colonnes supprimées : ['groupe_id', 'waypoint_id_1', 'waypoint_id_2', 'waypoint_id_3', 'waypoint_id_4', 'waypoint_id_5', 'waypoint_id_6', 'waypoint_id_7', 'groupe_id_1', 'groupe_id_2', 'groupe_id_3', 'groupe_id_4', 'groupe_id_5', 'groupe_id_6', 'groupe_id_7', 'groupe_unique_id_1', 'groupe_unique_id_2', 'groupe_unique_id_3', 'groupe_unique_id_4', 'groupe_unique_id_5', 'groupe_unique_id_6', 'groupe_unique_id_7']
                       0             1
mission_id  1.000000e+00  1.000000e+00
time_1      1.745179e+09  1.745179e+09
time_2      1.745179e+09  1.745179e+09
time_3      1.745179e+09  1.745179e+09
time_4      1.745179e+09  1.745179e+09
...                  ...           ...
snow_4      1.970000e-01  1.970000e-01
snow_5      1.970000e-01  1.970000e-01
snow_6      1.970000e-01  1.970000e-01
snow_7      1.970000e-01  1.970000e-01
is_attack   0.000000e+00  0.000000e+00

[170 rows x 2 columns]


In [145]:
import pandas as pd

df = pd.read_csv("vols_groupes_par_7.csv")

target_x_cols = [f"target_x_{i}" for i in range(1, 8)]

def detect_attack(row):
    for col in target_x_cols:
        val = row[col]
        # Si la valeur n’est pas un entier (ex: 5.1), c’est pas une attaque
        if not float(val).is_integer():
            return 0
    return 1  # Si toutes les valeurs sont entières

df['is_attack'] = df.apply(detect_attack, axis=1)

df.to_csv("vols_groupes_mis.csv", index=False)
print("✅ is_attack = 1 uniquement si toutes les target_x_i sont des entiers.")


✅ is_attack = 1 uniquement si toutes les target_x_i sont des entiers.


In [146]:
g=pd.read_csv('vols_groupes_mis.csv')
g['is_attack'].unique()

array([1, 0])

In [147]:
import pandas as pd

# Charger le fichier aplati
df_flat = pd.read_csv("vols_groupes_mis.csv")

# Liste des colonnes de base à extraire (sans le suffixe _1, _2, _3)
base_cols = [
    "mission_id", "time", "real_x", "real_y", "real_z",
    "target_x", "target_y", "target_z",
    "vx", "vy", "vz", "ax", "ay", "az",
    "roll", "pitch", "yaw",
    "wind_x", "wind_y", "wind_z",
    "erreur_m", "velocity", "rain", "fog", "snow"
]

# Stocker les lignes reconstituées
rows = []

# Pour chaque ligne du fichier aplati (correspond à un groupe)
for _, row in df_flat.iterrows():
    mission_id = row["mission_id"]
    is_attack = row["is_attack"]
    
    for i in range(1, 8):  # jusqu'à 7 waypoints
        waypoint = {"mission_id": mission_id, "is_attack": is_attack}
        valid = False
        
        for col in base_cols:
            col_name = f"{col}_{i}"
            if col_name in row:
                waypoint[col] = row[col_name]
                valid = True
        
        if valid:
            rows.append(waypoint)

# Convertir en DataFrame
df_unflattened = pd.DataFrame(rows)

# Réorganiser les colonnes pour correspondre à l'ordre voulu
final_columns = [
    "mission_id", "time", "real_x", "real_y", "real_z",
    "vx", "vy", "vz", "ax", "ay", "az",
    "roll", "pitch", "yaw",
    "wind_x", "wind_y", "wind_z",
    "erreur_m", "velocity", "rain", "fog", "snow",
    "is_attack"
]
df_unflattened = df_unflattened[final_columns]

# Réinitialiser l'index
df_unflattened = df_unflattened.reset_index(drop=True)

# Sauvegarder le résultat
df_unflattened.to_csv("vols_reconstruits_filtrés.csv", index=False)

print(f"✅ Données désaplaties et filtrées avec succès : {len(df_unflattened)} waypoints sauvegardés.")
print(df_unflattened.head(5))

✅ Données désaplaties et filtrées avec succès : 5607 waypoints sauvegardés.
   mission_id          time  real_x  real_y  real_z        vx        vy  \
0           1  1.745179e+09     0.0     0.0    -5.0  0.274891 -0.002897   
1           1  1.745179e+09     2.0     0.0    -5.0  0.274891 -0.002897   
2           1  1.745179e+09     4.0     0.0    -5.0  0.274891 -0.002897   
3           1  1.745179e+09     6.0     0.0    -5.0  0.274891 -0.002897   
4           1  1.745179e+09     8.0     0.0    -5.0  0.274891 -0.002897   

         vz        ax        ay  ...       yaw    wind_x    wind_y    wind_z  \
0  0.334512 -0.051712  0.018813  ... -3.119629  2.709812  1.738482  0.427165   
1  0.334512 -0.088959 -0.030237  ... -3.119629  3.098014  1.627995  0.498919   
2  0.334512 -0.088959 -0.030237  ... -3.119629  3.296738  1.796560  0.459864   
3  0.334512 -0.077355 -0.099897  ... -3.119636  2.927465  2.284593  0.499880   
4  0.334512 -0.156777 -0.017914  ... -3.119636  2.929180  1.724429  0.494

In [148]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("donnees_vols_valides_Rate.csv")

# Réinitialiser l'index
df = df.reset_index(drop=True)

# Garder la valeur is_attack originale par mission_id
is_attack_values = df.groupby('mission_id')['is_attack'].first().reset_index()

# Ajouter un compteur par mission_id (position du waypoint)
df['ligne_id'] = df.groupby('mission_id').cumcount()

# Passer de long à wide : chaque ligne devient un vol à plat
df_flat = df.pivot(index='mission_id', columns='ligne_id')

# Aplatir les colonnes multi-niveaux
df_flat.columns = [f"{col}_{i}" for col, i in df_flat.columns]

# Réinitialiser l'index pour fusionner
df_flat = df_flat.reset_index()

# Fusionner avec les valeurs originales de is_attack
df_flat = pd.merge(df_flat, is_attack_values, on='mission_id', how='left')

# Supprimer toutes les colonnes is_attack_X qui ont été créées par le pivot
cols_to_drop = [col for col in df_flat.columns if col.startswith('is_attack_')]
df_flat = df_flat.drop(columns=cols_to_drop)

# Sauvegarder le fichier aplati
df_flat.to_csv("vols_aplatis_R.csv", index=False)

print("✅ Vols aplatis avec succès ! (avec un seul is_attack par vol)")

✅ Vols aplatis avec succès ! (avec un seul is_attack par vol)


In [149]:

print(df.isna().any().any())
colonnes_avec_nan = df.columns[df.isna().any()].tolist()

# Afficher les colonnes concernées
print("Colonnes contenant des NaN :")
print(colonnes_avec_nan)

# Afficher le nombre de NaN par colonne
print("\nNombre de NaN par colonne :")
print(df[colonnes_avec_nan].isna().sum())

True
Colonnes contenant des NaN :
['snow']

Nombre de NaN par colonne :
snow    735
dtype: int64
